In [1]:
import numpy as np
import qiskit as qk
from qiskit import Aer
%matplotlib inline

In [2]:
x = np.array([[1,0,1,0]])

In [ ]:
backend = Aer.get_backend("statevector_simulator")

def basis_encoding(x):
    M,N = x.shape
    clas_reg = qk.QuantumRegister(N)
    loading_reg = qk.QuantumRegister(N)
    storage_reg = qk.QuantumRegister(N)
    ancillae = qk.QuantumRegister(2)
    
    circuit = qk.QuantumCircuit(classical_reg, loading_reg, storage_reg, ancillae)
    
    for i in range(M):
        circuit.cx(x[i], loading_reg)

In [ ]:
basis_encoding(x)

In [6]:
reg1 = qk.QuantumRegister(3)
circuit2 = qk.QuantumCircuit(reg1)
circuit2.h(reg1)
print(reg1)

QuantumRegister(3, 'q3')
